In [23]:
import openai
import os
from IPython.display import Markdown

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

def ask_gpt(model, system_prompt, question, conversation_history=None, max_history=5):
    if conversation_history is None:
        conversation_history = []

    # Truncate conversation_history if it exceeds the limit
    if len(conversation_history) > max_history:
        conversation_history = conversation_history[-max_history:]

    conversation = [
        {"role": "system", "content": system_prompt},
        *conversation_history,
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=conversation,
        max_tokens=150,                 # one token is roughly 4 characters
        temperature=1.0,
        top_p=1,
    )

    assistant_response = response['choices'][0]['message']['content']
    
    # Update the conversation history
    conversation_history.append({"role": "assistant", "content": assistant_response})
    conversation_history.append({"role": "user", "content": question})

    return assistant_response, conversation_history


In [2]:

# Usage example
model_name = "gpt-3.5-turbo"
# model_name = "gpt-4"
system_prompt = "You are a helpful assistant that provides new parents with baby name suggestions."

question = "Can you give me some categories of names that I could investigate?"
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)


Sure, here are some categories of names that you may want to explore:

1. Classic names: These are names that have been in use for generations and have a timeless feel. Examples include Elizabeth, William, Catherine, and James.

2. Nature-inspired names: These names draw inspiration from the natural world, such as River, Rose, or Skylar.

3. Modern names: These names tend to be more recent and reflect current trends. Examples include Luna, Maverick, Aria, and Grayson.

4. Names with meaningful origins: Some parents choose names based on their meanings or origins, such as Gabriel (Hebrew for "God is my strength") or Ava (Latin for "life").

5. Unisex names: These names


In [6]:
question = "Can you suggest me a list of 10 boys names which are both classic and nature inspired?"
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Sure, here are 10 classic and nature-inspired names for boys:

1. Jasper - inspired by the name of a gemstone and also after the beautiful natural park in Canada.
2. Forrest - means "woodland" and is a classic name for boys.
3. Phoenix - inspired by the mythical bird and is also the name of a city in Arizona.
4. Asher - a biblical name that means "happy" or "blessed" and also refers to trees or groves of trees.
5. River - a name that is inspired by natural waterways and is also a popular unisex name.
6. Orion - inspired by the constellation named after a hunter in Greek mythology.
7. Leo - meaning "lion", a


In [7]:
question = """ This is my list of favourite boys names: 
---
Adam, Felix, Odin, Membrillo
---
Can you please give me 10 more boys names that are similar to these?
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Absolutely! Here are 10 more boys names that are similar to the names on your list:

1. Leo
2. Jasper
3. Silas
4. Caspian
5. August
6. Ezra
7. Atlas
8. Phoenix
9. Leonidas
10. Orion

I hope these suggestions are helpful in finding the perfect name for your baby boy!


In [8]:
question = """ User: Can you make them sound more earthy?
---
[System] Provide a list of 10 boys names
---
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Sure! Here are 10 earthy boy names to consider:

1. Jasper
2. Orion
3. Sage
4. Asher
5. Cedar
6. Terra
7. River
8. Forrest
9. Phoenix
10. Canyon


In [3]:
question = """ User: I like the name Remi, can you convince me using all the best reasons why it's a great name?
"""
response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

Absolutely! Here are some reasons why Remi is a fantastic name for your baby:

1. Unique and Modern: Remi is a trendy and unique name with a modern feel. It's a great pick for parents who want a name that's not too common, but also not too out there.

2. Easy to Pronounce: Remi is a simple and easy-to-pronounce name, which is important for your child's schooling and other activities.

3. Gender-Neutral: Remi is a gender-neutral name, which means it's a great choice for parents who want a name that can be used for either a boy or a girl.

4. A Great Meaning: The name Remi has French roots and means "oarsman


In [22]:
question = """
[System Instructions]
Always bold the names using **double asterisks**.
    For example: **Remi** is a great name, as is **Sophie**.
Use bullet points for each reason.
Do not begin with an introduction, just start with the first reason.
---
 User: I like the name Remi, can you convince me it's a better name (for me) than my second choice: Sophie?

"""
model_name='gpt-4'
response, convo_history = ask_gpt(model_name, system_prompt, question)
Markdown(response)

- **Remi** is a more unique and less common name, giving your child a sense of individuality, while **Sophie** is a popular name that many other children may share.
- **Remi** is a gender-neutral name, allowing for more flexibility and personal identity exploration, whereas **Sophie** is traditionally seen as a feminine name.
- **Remi** has a modern and sophisticated sound, making it a great choice for parents looking for a name that feels fresh and current, while **Sophie** has a more classic and timeless feel.
- **Remi** is easier to pronounce and spell, which can make life simpler for your child in various situations, while **Sophie** may require more corrections in terms of

In [1]:
import os
import openai
import discord
import nest_asyncio
from discord.ext import commands
from discord.ui import Button, View
from IPython.display import Markdown

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

def ask_gpt(model, system_prompt, question, conversation_history=None, max_history=5):
    if conversation_history is None:
        conversation_history = []

    # Truncate conversation_history if it exceeds the limit
    if len(conversation_history) > max_history:
        conversation_history = conversation_history[-max_history:]

    conversation = [
        {"role": "system", "content": system_prompt},
        *conversation_history,
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=conversation,
        max_tokens=150,                 # one token is roughly 4 characters
        temperature=1.0,
        top_p=1,
    )

    assistant_response = response['choices'][0]['message']['content']
    
    # Update the conversation history
    conversation_history.append({"role": "user", "content": question})
    conversation_history.append({"role": "assistant", "content": assistant_response})

    # log to file
    file = 'logs/conversation.txt'
    # don't append
    with open(file, 'w') as f:
        # list to string
        conversation_str = '\n'.join([f"{x['role']}: {x['content']}\n" for x in conversation_history])
        f.write(conversation_str)
    
    return assistant_response, conversation_history


# Apply the nest_asyncio patch
nest_asyncio.apply()

DISCORD_TOKEN = os.getenv('DISCORD_TOKEN_NAMEBABY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TARGET_CHANNEL_ID = int(os.getenv('DISCORD_CHANNEL_ID_NAMEBABY'))
openai.api_key = OPENAI_API_KEY

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True


class Buttons(discord.ui.View):
        def __init__(self, *, timeout=180):
            super().__init__(timeout=timeout)

        @discord.ui.button(label="clickme",style=discord.ButtonStyle.green)
        async def my_button(self, button:discord.ui.Button, interaction:discord.Interaction):
            await interaction.response.send_message("You clicked me!")

class Clickme(commands.Cog):
    def __init__(self, bot):
        self.bot = bot
    
    @commands.Cog.listener()
    async def on_ready(self):
        print('Clickme cog loaded')
    
    @commands.command()
    async def click(self, ctx):
        await ctx.send("Message with a button", view=Buttons())
    
async def setup(bot):
    await bot.add_cog(Clickme(bot))


class ChatBot(discord.Client):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = "gpt-3.5-turbo"
        self.system_prompt = """You are a helpful assistant that provides new parents with baby name suggestions.
                                Include an emoji at the end of your message"""
        self.conversation_history = [] 
       
    async def on_ready(self):
        print(f'We have logged in as {self.user}')
        target_channel = self.get_channel(TARGET_CHANNEL_ID)
        if target_channel:

            # ============= Add Buttons ============= #
            button1 = Button(label="Start", style=discord.ButtonStyle.green, emoji="👋")
            async def button1_callback(interaction):
                # edit button to add "working" emoji
                await interaction.response.defer()
                await interaction.channel.send("/ask Can you give me a list of 10 trendy baby girl names with a scottish influence")
            button1.callback = button1_callback

            view = View()
            view.add_item(button1)

            await target_channel.send("`namebaby` is online  💅", view=view)
        else:
            print("Could not find the target channel.")

    async def on_message(self, message):
        if (message.author == self.user):
            if message.content.startswith("/ask"):
                message.content = message.content[4:]
                pass # allow the bot to respond to its own `/ask` messages
            else: return # else ignore the message

        print(f"incoming: {message.content}")

        if message.content.startswith("/help") or message.content.startswith("/info"):
            # respond with current config
            await message.channel.send(f"**namebaby Config**\nModel: {self.model}\n")
        
        elif message.content.startswith("/test"):
            await message.channel.send('test successful', view=Buttons())
            
        else:
            assistant_response, self.conversation_history = ask_gpt(
                model=self.model,
                system_prompt=self.system_prompt,
                question=message.content,
                conversation_history=self.conversation_history,
                max_history=5) 
            
            print(f"response: {assistant_response}")

            # ============= Add Buttons ============= #
            button1 = Button(label="+10 More", style=discord.ButtonStyle.green, emoji="👋")
            async def button1_callback(interaction):
                await interaction.response.defer()
                await interaction.channel.send("/ask please provide 10 more names")
            button1.callback = button1_callback

            vb = View()
            vb.add_item(button1)
            button2 = Button(label="Change Influence", emoji="🌐")
            async def button2_callback(interaction):
                await interaction.response.defer()
                await interaction.channel.send("button 2 followup", view=vb)
            button2.callback = button2_callback

            view = View()
            view.add_item(button1)
            view.add_item(button2)

            await message.channel.send(assistant_response, view=view)


client = ChatBot(intents=intents)
await client.start(DISCORD_TOKEN)

We have logged in as namebaby#4590
incoming:  Can you give me a list of 10 trendy baby girl names with a scottish influence
response: Sure, here are 10 trendy baby girl names with Scottish influence:
1. Ainsley 
2. Isla 
3. Eilidh 
4. Skye 
5. Freya 
6. Greer 
7. fíona 
8. Maisie 
9. Lachlan 
10. Elsie 

👶🏼💕
incoming:  please provide 10 more names
response: Absolutely, here are 10 more trendy baby girl names with Scottish influence:
1. Adair
2. Alba
3. Caledonia 
4. Finley
5. Mairi 
6. Rhona 
7. Rowan 
8. Sorcha 
9. Tegan 
10. Edina 

👶🏻💖
